# Computing the sensitivities of eigenvalues in linear elasticity:
# An example with OptimiSM

## Introduction

This notebook demonstrates spectral analysis of a linear elastic body, and the computation of sensitivities of the fundamental natural frequency to selected design parameters. The sensitivity algorithm could be used, for example, to drive a design optimization program.

To keep things simple, the body is chosen to be a cantilevered plate. The design variable is the density, first treated as a scalar value for the entire plate, and then considered as an $L^2$ field that can take piecewise constant values over each finite element.

In [5]:
from functools import partial
import jax
import jax.numpy as np
from scipy.sparse import linalg
from sksparse import cholmod

from optimism import Mesh
from optimism.FunctionSpace import DofManager, EssentialBC
from optimism import FunctionSpace
from optimism import Mechanics
from optimism.material import LinearElastic
from optimism import Objective
from optimism import VTKWriter
from optimism import QuadratureRule
from optimism import SparseMatrixAssembler

First, we set up the geometry. The body is a cantilevered plate, which will be assumed to deform in plane strain conditions. We could read in a mesh in the EXODUS/GENESIS format (the native format of Cubit), but for this problem, it's easy enough to generate a structured mesh by hand. We'll construct a plate of thickess $h = 1$ and span $L = 10$, and then create a nodeset on the bottom edge to apply the cantilever boundary conditions. We'll also make the mesh use quadratic elements.

In [6]:
h = 1.0
L = 10.0
mesh = Mesh.construct_structured_mesh(Nx=4, Ny=40, xExtent=[0.0, h], yExtent=[0.0, L], elementOrder=2)
nodeTol = 1e-8
nodeSets = {'bottom': np.flatnonzero(mesh.coords[:,1] < 0 + nodeTol)}
mesh = Mesh.mesh_with_nodesets(mesh, nodeSets)

The resulting mesh looks like this:

![mesh](images/mesh.png)

We'll create a quadrature rule that is capable of exactly integrating polynomials up to 4th degree, so that the mass matrix will be exact (the stiffness matrix integraes lower degree polynomials, so it will be exact as well). Then we'll create our discrete function space over our mesh. The function space discovers which degree interpolant it should use from the `mesh` object.

In [7]:
quadRule = QuadratureRule.create_quadrature_rule_on_triangle(degree=4)
fs = FunctionSpace.construct_function_space(mesh, quadRule)

The cantilever boundary conditions are applied as follows:

In [8]:
ebcs = [EssentialBC(nodeSet='bottom', component=0),
        EssentialBC(nodeSet='bottom', component=1)]

Then we create a `dofManager` object, which aids tracking of degrees of freedom that are constrained with essential boundary condtions.

In [9]:
dofManager = DofManager(fs, dim=2, EssentialBCs=ebcs)

Now, let's instantiate the material model. The properties are not important for the purpose of this example, so we'll schoose them arbitrarily.

In [10]:
E = 5.0
nu = 0.25
rho = 1.0
props = {'elastic modulus': E,
         'poisson ratio': nu,
         'density': rho}
material = LinearElastic.create_material_model_functions(props)

We instantiate the physics modules, which provide methods for computing energies, residuals, and related operations.

In [11]:
solidStatics = Mechanics.create_mechanics_functions(fs, 'plane strain', material)
solidDynamics = Mechanics.create_dynamics_functions(fs, 'plane strain', material, Mechanics.NewmarkParameters)

## Spectral analysis

SciPy conveniently provides the symmetric sparse matrix eigensolver interface `eigsh` in the `scipy.sparse` package.  (Under the hood, this calls down to an interative Lanczos method in ARPACK). 
There are several possiblities to code our finite element methods to this interface. The simplest way is to explicitly assemble the stiffness and mass matrices in sparse matrix data structures and pass them to the solver directly.
Another option is to provide functions that give the action of the system matrices on a vector using the finite element assembly operators. For now, let's keep the explanation simple and choose the matrix route. Later, we'll demonstrate how to get the implicit matrix action representation and will show that it's also easy to do.

Let's define functions to assemble the stiffness and mass matrices in a SciPy sparse matrix (compressed sparse row format). There are methods in the physics module that do the work for us:

In [12]:
def assemble_sparse_stiffness_matrix(Uu, p):
    U = dofManager.create_field(Uu)
    internalVariables = p.state_data
    elementStiffnesses = solidStatics.compute_element_stiffnesses(U, internalVariables)
    return SparseMatrixAssembler.assemble_sparse_stiffness_matrix(
        elementStiffnesses, mesh.conns, dofManager)

def assemble_sparse_mass_matrix():
    elementMasses = solidDynamics.compute_element_masses()
    return SparseMatrixAssembler.assemble_sparse_stiffness_matrix(
        elementMasses, mesh.conns, dofManager)

The generalized eigenvalue problem for vibrations is usually written in the form
\begin{equation}
    \mathbf{K} \mathbf{v}_i = \lambda_i \mathbf{M} \mathbf{v}_i,
\end{equation}
where $\mathbf{K}$ is the stiffness matrix, $\mathbf{M}$ is the mass matrix, $\lambda_i$ are the squares of the natural (circular) frequencies, and $\mathbf{v}_i$ are the corresponding eigenmodes.
We want to search for the smallest eigenvalue of the system, since that is often the critical resonance of interest in applications. However, iterative eigensolvers are most efficient in searching for the *largest* eigenvalues. There are a number of methods to deal with this problem; we will approach it by defining the reciprocal frequencies $\mu_i := \lambda_i^{-1}$ and rewriting the eigenproblem as
\begin{equation}
    \mathbf{M} \mathbf{v}_i = \mu_i \mathbf{K} \mathbf{v}_i.
\end{equation}
We then search for the $n$ largest $\mu$-eigenvalues (let's say $n=10$ to be concrete).
In this mode of operation, `eigsh` needs the action of the inverse of the stiffness matrix. 
Since we've already chosen to explicitly form $\mathbf{K}$, we might as well use a Cholesky factorization for this purpose.
We use the `cholmod` module in `sksparse` (which is a wrapper for the well-known CHOLDMOD package from SuiteSparse).

In [16]:
Uu = np.zeros(dofManager.get_unknown_size()) # set displacement field to zero
state = solidStatics.compute_initial_state() # intialize internal state variables (this material has none)
p = Objective.Params(None, state)
K = assemble_sparse_stiffness_matrix(Uu, p)
M = assemble_sparse_mass_matrix()
    
KFactor = cholmod.cholesky(K)
Kinv = linalg.LinearOperator(K.shape, KFactor)
    
nModes = 10
mu, modes = linalg.eigsh(A=M, k=nModes, M=K, Minv=Kinv, which='LA')

Now we have our eigenvalues and eigenvectors. Recall that we really want the reciprocals of these, so let's do that operation. The eigenvalues are returned in ascending order, so the fundamental eigenvalue is actually going to be the *last* one in the list (in NumPy terms, the "-1" index). We'll also grab the fundamental mode and call it `Vu0`. (Note: `lamda` isn't a typo, it's avoiding a collision with the Python `lambda` keyword.)

In [17]:
lamda = 1/mu
lamda0 = lamda[-1]
Vu0 = modes[:, -1]

Let's write out the mode to a VTK file so we can inspect it:

In [18]:
writer = VTKWriter.VTKWriter(mesh, baseFileName='beam-vibration')
V0 = dofManager.create_field(Vu0)
writer.add_nodal_field('mode_0', V0, VTKWriter.VTKFieldType.VECTORS)
writer.write()

This writes the results to the file `beam-vibration.vtk`. Here is an animation of the mode created in ParaView:

![](images/mode0.gif)

The fundamental mode looks as expected - it's the quasi-static bending mode.
The exact solution for the fundamental natrual frequency is
\begin{equation}
    \omega_0 = 1.875^2 \sqrt{\frac{Eh^2}{12(1 - \nu^2) \rho L^4}}.
\end{equation}
Let's compare this with the numerical answer.

In [19]:
exactNaturalFrequency = 1.875**2*np.sqrt(E/(1-nu**2)*(h**2/12)/rho/L**4)
print('exact natural frequency: ', exactNaturalFrequency)
print('numerical natural frequency:', np.sqrt(lamda0))

exact natural frequency:  0.0234375
numerical natural frequency: 0.023298050290143358


This is an error of less than 1% - not bad! If we wish, we could decrease the error with mesh refinement.

## Computing sensitivity of an eigenvalue to a scalar parameter

Imagine now the system physics are parameterized by some design variable.
We wish to find the sensitivity of any particular eigenvalue to this parameter.
To keep things simple, we will assume that the eigenvalue is not repeated.
Differentiating the generalized eigenproblem yields

\begin{equation*}
    \mathbf{K}' \mathbf{v}_i + \mathbf{K} \mathbf{v}'_i =
    \lambda'_i \mathbf{M} \mathbf{v}_i + \lambda_i \mathbf{M}' \mathbf{v}_i + \lambda_i \mathbf{M}_i \mathbf{v}'_i,
\end{equation*}

where the prime symbol indicates differentiation with respect to the parameter.
We can multiply both sides on the left by the eigenvector to get

\begin{equation*}
    \mathbf{v}^T_i \mathbf{K}' \mathbf{v}_i + \mathbf{v}^T_i \mathbf{K} \mathbf{v}'_i =
    \lambda'_i \mathbf{v}^T_i \mathbf{M} \mathbf{v}_i + \lambda_i \mathbf{v}^T_i \mathbf{M}' \mathbf{v}_i 
    + \lambda_i \mathbf{v}^T_i \mathbf{M}_i \mathbf{v}'_i.
\end{equation*}

By moving terms around and using transpose operations, we can rewrite this as

\begin{equation*}
    \lambda'_i \mathbf{v}^T_i \mathbf{M} \mathbf{v}_i = 
    \mathbf{v}^T_i \mathbf{K}' \mathbf{v}_i
    - \lambda_i \mathbf{v}^T_i \mathbf{M}' \mathbf{v}_i 
    + (\mathbf{v}'_i)^T \left( \mathbf{K}^T \mathbf{v}_i - \lambda_i \mathbf{M}^T \mathbf{v}_i \right).
\end{equation*}

But recall that $\mathbf{K}$ and $\mathbf{M}$ are symmetric; hence the expression in parentheses is $\mathbf{K} \mathbf{v}_i - \lambda_i \mathbf{M} \mathbf{v}_i$, which vanishes, because $\lambda_i$ and $\mathbf{v}_i$ together satisfy the generalized eigenvalue problem.
Hence the sought sensitivity expression is

\begin{equation}
    \lambda_i' = 
    \frac{\mathbf{v}_i^T \left( \mathbf{K}' - \lambda_i \mathbf{M}' \right) \mathbf{v}_i}{\mathbf{v}_i^T \mathbf{M} \mathbf{v}_i}.
    \tag{3}
\end{equation}

Let's start with a simple sensitivity problem.
We will take the density $\rho$ to be the design parameter and compute the derivative $\frac{d \lambda_0}{d \rho}$, where the density is taken to be spatially homogeneous (that is, a single value for the whole body).
We can compute the answer for this analytically, making it a useful verification problem.

It's clear that only the mass matrix depends on this parameter. We'll pretend we don't know this to emulate the more general case where both $\mathbf{K}$ and $\mathbf{M}$ depend on the parameter.
To implement the sensitivity embodied in equation (3), we implement functions for the potential energy and the kinetic energy:

In [20]:
def compute_potential_energy(Uu, p):
    U = dofManager.create_field(Uu)
    internalVariables = p[1]
    return solidStatics.compute_strain_energy(U, internalVariables)
    
def compute_parameterized_kinetic_energy(Vu, density):
    def lagrangian_density(w, gradw, q, x, dt):
        return 0.5*density*np.dot(w, w)
    
    V = dofManager.create_field(Vu)
    dtUnused = 0.0
    return FunctionSpace.integrate_over_block(fs, V, state, dtUnused, lagrangian_density, mesh.blocks['block_0'])

Conveniently, we can use the `compute_strain_energy()` method provided by the `solidStatics` object. There is a also a function for computing the kinetic energy in the `solidDynamics` object, but notice we didn't use it. We re-implmented this function to expose `density` as an argument so that we can differentiate with respect to it.

The stiffness matrix is the discrete representation of the second variation of the potential energy functional. We can obtain the action of this operator by taking derivatives with automatic differentiation. Let's take the first derivative of the potential energy with respect to displacement, which is the residual operator (or its negative, depending on your preferred sign convention).

In [21]:
residual = jax.jacrev(partial(compute_potential_energy, p=p), 0)

This derivative was taken with reverse mode (an adjoint derivative), which is the appropriate choice, since we are taking the derivative of a scalar function with respect to many nodal degrees of freedom. Note that `jacrev` produces a *function* that takes the same arguments as the primal function. (The `partial` is a partial function evalutation, just fixing the internal state variable argument so we don't need to keep passing the same value in repeatedly.)

The action of the stiffness matrix on a vector of nodal unknowns, $\mathbf{w} \mapsto \mathbf{K} \mathbf{w}$, is the directional derivative of the residual in the direction $\mathbf{w}$. We can use another Jax method for this, `jvp` (for "Jacobian-vector product"):

In [22]:
def stiffness_action(W):
    return jax.jvp(residual, (Uu,), (W,))[1]

This linearizes `residual` at the argument `Uu` in the direction `W`. The `[1]` index gets the derivative; taking `[0]` would get the value of the function.

We could do similar AD operations on the kinetic energy function to get the action of the mass matrix. But since we're the term we're trying to assemble is a quadratic form, and the kinetic energy is *already* a quadratic form, we can make use of the kinetic energy function directly. That is, we will compute the mass term $\lambda_0 \mathbf{v}_0^T \mathbf{M} \mathbf{v}_0$ as
\begin{equation*}
    \lambda_0 \mathbf{v}_0^T \mathbf{M} \mathbf{v}_0 = 2 \lambda_0 T(\boldsymbol{v}_0),
\end{equation*}
where $T$ is the kinetic energy functional
\begin{equation*}
    T(\boldsymbol{w}) = \int_{\Omega} \frac{1}{2} \rho \boldsymbol{w} \cdot \boldsymbol{w} \, dv.
\end{equation*}

To compute the sensitivity in equation (3), we will define the function
\begin{equation*}
    Q(\rho) = \frac{\mathbf{v}_0^T \left( \mathbf{K}(\rho) - \lambda_0 \mathbf{M}(\rho) \right) \mathbf{v}}{\mathbf{v}_0^T \mathbf{M} \mathbf{v}_0},
\end{equation*}
and then differentiate it. Note that the $\mathbf{v}_0$, $\lambda_0$, and the denominator of this function are regarded as contants.

In [23]:
rayleighDenominator = np.dot(Vu0, M@Vu0)

def Q(density):
    numerator = np.dot(Vu0, stiffness_action(Vu0)) - 2*lamda0*compute_parameterized_kinetic_energy(Vu0, density)
    return numerator / rayleighDenominator

lamdaPrime = jax.grad(Q)(rho)
print(lamdaPrime)

-0.0005427991473220492


To verify this, let's compare with the exact solution. Using the square of equation (1), we have
\begin{equation*}
    \frac{d \lambda_0}{d \rho} = - \frac{\lambda_0}{\rho}
    \tag{2}
\end{equation*}


In [24]:
exactLamdaPrime = -exactNaturalFrequency**2/rho
print(exactLamdaPrime)

-0.00054931640625


Our work checks out: this is only an error of about 1% (again due to the discretization).

## Computing sensitivity of an eigenvalue to a field

Let's make the sensitivity problem more interesting. Let the density now be a piecewise constant field that can take on different values on each finite element. OptimiSM has facilities for this: the `integrate_over_block` method accepts variadic arguments at the end to pass as parameters to the functions it integrates. We'll use this to parameterize the kinetic energy with the element-wise density values:

In [25]:
def compute_parameterized_kinetic_energy2(Vu, internalVars, densities):
    V = dofManager.create_field(Vu)
    def lagrangian_density(u, gradu, Q, X, dt, density):
        return Mechanics.kinetic_energy_density(u, density)
    dtUnused = 0.0
    return FunctionSpace.integrate_over_block(fs, V, internalVars, dtUnused, lagrangian_density, mesh.blocks['block_0'], densities)

def Q2(densities):
    numerator = np.dot(Vu0, stiffness_action(Vu0)) - 2*lamda0*compute_parameterized_kinetic_energy2(Vu0, state, densities)
    return numerator / rayleighDenominator

Then take a reverse-mode derivative of the `Q2` functional to get a function that yields the sensitivities of the eigenvalue to each element density value:

In [26]:
compute_eigenvalue_sensitivity = jax.jacrev(Q2)

As a verification measure, we can take the sensitivity of the eigenvalue to a uniform change in the density field and check that it is equal to the sensitivity of the eigenvalue to the uniform density computed above:

In [27]:
rhoField = rho*np.ones(Mesh.num_elements(mesh))
lPrime = compute_eigenvalue_sensitivity(rhoField)
print(np.sum(lPrime))

-0.0005427991473220492


This checks out with the result above. Now let's write this computed sensitivity field to the VTK file (recall that it is `beam-vibration.vtk`):

In [28]:
writer.add_cell_field('eigenvalue_sensitivity', lPrime, VTKWriter.VTKFieldType.SCALARS)
writer.write()

If you open the file in ParaView and visualize the `eigenvalue_sensitivity` field, it should look like this:

![eigenvalue sensitivity field](images/eigenvalue_sensitivity_field.bmp)

This field seems to make sense; the sensitivity is zero at the root of the plate where the constraints keep it stationary, and increase in magnitude as you move towards the free end. The sign of the sensitivity is negative everywhere, since a marginal increase in mass retards the motion and depresses the vibration frequency.

This technique can be extended to nonlinear problems with the AD capabilities furnished by Jax and embedded in OptimiSM. We illustrate this in the next example.